In [1]:
# imports, parameters, environment, etc.
import torch
import torch.nn as nn
from utilities.ogbench_utilities import *
from decision_diffuser import DecisionDiffuserTrainConfig
from janner_diffuser import JannerDiffuserTrainConfig
from decision_diffuser import eval_model as dd_eval
from janner_diffuser import eval_model as janner_eval
from utilities.cleandiffuser.invdynamic import MlpInvDynamic
from utilities.cleandiffuser.diffusion import ContinuousDiffusionSDE
from utilities.cleandiffuser.diffusion import DiscreteDiffusionSDE
from utilities.cleandiffuser.nn_diffusion import JannerUNet1d
from utilities.cleandiffuser.nn_condition import MLPCondition
from ogbench import make_env_and_datasets
import matplotlib.pyplot as plt
import glob
env = 'scene-play-v0'
# env = 'cube-quadruple-play-v0'
# env = 'pointmaze-giant-stitch-v0'
dir = 'trained_models'
memory = 20
horizon = 1024
num_envs = 1
num_episodes = 1
temperature = 0.5
n_exec_steps = 10
# if doing flatland stuff
flatland = False
n_dims = 2

In [ ]:
### eval for diffuser w/inpainting

# loads config, modifies to work w/given parameters
config = JannerDiffuserTrainConfig()
config.memory = memory
config.horizon = horizon
config.num_envs = num_envs
config.num_episodes = num_episodes
config.n_exec_steps = n_exec_steps
config.env = env
config.inpaint = True

# loads environment
envs = make_envs_and_datasets(config.env,compact_dataset=False,num_envs = config.num_envs,env_only=True)
obs_dim, act_dim = envs.single_observation_space.shape[0], envs.single_action_space.shape[0]

# loads inverse dynamics
invdyn = MlpInvDynamic(obs_dim, act_dim, 512, nn.Tanh(), {"lr": 2e-4}, device=config.device)
invdyn.load(config.invdyn_path)
invdyn.eval()

# loads agent 
# --------------- Network Architecture -----------------# 
nn_diffusion = JannerUNet1d(
        obs_dim, model_dim=config.model_dim, emb_dim=config.model_dim, dim_mult=[1, 2, 2, 2],
        timestep_emb_type="positional", attention=False, kernel_size=5)


# ----------------- Masking -------------------
# NOTE - not sure what any of this does (should figure out!)
fix_mask = torch.zeros((config.horizon, obs_dim)) 
fix_mask[0, :] = 1. # for the starting/current state
loss_weight = torch.ones((config.horizon, obs_dim))

# --------------- difuse --------------------
agent = DiscreteDiffusionSDE(
        nn_diffusion, None,
        fix_mask=fix_mask, loss_weight=loss_weight, classifier=None, ema_rate=config.ema_rate,
        device=config.device, diffusion_steps=config.diffusion_steps, predict_noise=config.predict_noise)
name = 'JannerInpaint'
agent.load(glob.glob(dir + '/' + name + '*' + env + '*' + 'latest.pt')[0])

# Environment info 
print('==== Environment Info ====')
print('Name:',config.env)
print('Obs Space:',obs_dim)
print('Act Space:',act_dim)
# main evaluation loop
print('==== Running Eval ====')
for task_id in [1,2,3,4,5]:
    print('== Task',task_id,' ==')
    envs.reset(options = {'task_id':task_id})
    avg_completion = janner_eval(agent,invdyn,envs,config)
    print('Avg. Completion:',avg_completion)
    

In [ ]:
### eval for decision diffuser base 

# loads config, modifies to work w/given parameters
config = DecisionDiffuserTrainConfig()
config.memory = memory
config.horizon = horizon
config.num_envs = num_envs
config.num_episodes = num_episodes
config.temperature = temperature
config.n_exec_steps = n_exec_steps
config.env = env

# loads environment
envs = make_envs_and_datasets(config.env,compact_dataset=False,num_envs = config.num_envs,env_only=True)
obs_dim, act_dim = envs.single_observation_space.shape[0], envs.single_action_space.shape[0]

# loads inverse dynamics
invdyn = MlpInvDynamic(obs_dim, act_dim, 512, nn.Tanh(), {"lr": 2e-4}, device=config.device)
invdyn.load(config.invdyn_path)
invdyn.eval()

# loads agent 
# --------------- Network Architecture -----------------# --------------- Network Architecture -----------------
nn_diffusion = JannerUNet1d(
    obs_dim, model_dim=config.model_dim, emb_dim=config.model_dim, dim_mult=[1, 2, 2, 2],
    timestep_emb_type="positional", attention=False, kernel_size=5)
nn_condition = MLPCondition(
    in_dim=obs_dim, out_dim=config.emb_dim, hidden_dims=[config.emb_dim, ], act=nn.SiLU(), dropout=config.label_dropout)


# ----------------- Masking -------------------
# NOTE - not sure what any of this does (should figure out!)
fix_mask = torch.zeros((config.horizon, obs_dim))
fix_mask[0] = 1. # for starting state
if config.inpaint:
    fix_mask[-1] = 1. # for ending goal state
loss_weight = torch.ones((config.horizon, obs_dim))
loss_weight[1] = config.next_obs_loss_weight

# --------------- Diffusion Model with Classifier-Free Guidance --------------------
agent = ContinuousDiffusionSDE(
    nn_diffusion, nn_condition,
    fix_mask=fix_mask, loss_weight=loss_weight, ema_rate=config.ema_rate,
    device=config.device, predict_noise=config.predict_noise, noise_schedule="linear")

name = 'CFGDiffuser'
agent.load(glob.glob(dir + '/' + name + '*' + env + '*' + 'latest.pt')[0])

# Environment info 
print('==== Environment Info ====')
print('Name:',config.env)
print('Obs Space:',obs_dim)
print('Act Space:',act_dim)
# main evaluation loop
print('==== Running Eval ====')
for task_id in [1,2,3,4,5]:
    print('== Task',task_id,' ==')
    envs.reset(options = {'task_id':task_id})
    avg_completion = dd_eval(agent,invdyn,envs,config)
    print('Avg. Completion:',avg_completion)
    

In [ ]:
### eval for conditioned diffuser

# loads config, modifies to work w/given parameters
config = DecisionDiffuserTrainConfig()
config.memory = memory
config.horizon = horizon
config.num_envs = num_envs
config.num_episodes = num_episodes
config.temperature = temperature
config.n_exec_steps = n_exec_steps
config.env = env
config.w_cfg = 0.0

# loads environment
envs = make_envs_and_datasets(config.env,compact_dataset=False,num_envs = config.num_envs,env_only=True)
obs_dim, act_dim = envs.single_observation_space.shape[0], envs.single_action_space.shape[0]

# loads inverse dynamics
invdyn = MlpInvDynamic(obs_dim, act_dim, 512, nn.Tanh(), {"lr": 2e-4}, device=config.device)
invdyn.load(config.invdyn_path)
invdyn.eval()

# loads agent 
# --------------- Network Architecture -----------------# --------------- Network Architecture -----------------
nn_diffusion = JannerUNet1d(
    obs_dim, model_dim=config.model_dim, emb_dim=config.model_dim, dim_mult=[1, 2, 2, 2],
    timestep_emb_type="positional", attention=False, kernel_size=5)
nn_condition = MLPCondition(
    in_dim=obs_dim, out_dim=config.emb_dim, hidden_dims=[config.emb_dim, ], act=nn.SiLU(), dropout=config.label_dropout)


# ----------------- Masking -------------------
# NOTE - not sure what any of this does (should figure out!)
fix_mask = torch.zeros((config.horizon, obs_dim))
fix_mask[0] = 1. # for starting state
if config.inpaint:
    fix_mask[-1] = 1. # for ending goal state
loss_weight = torch.ones((config.horizon, obs_dim))
loss_weight[1] = config.next_obs_loss_weight

# --------------- Diffusion Model with Classifier-Free Guidance --------------------
agent = ContinuousDiffusionSDE(
    nn_diffusion, nn_condition,
    fix_mask=fix_mask, loss_weight=loss_weight, ema_rate=config.ema_rate,
    device=config.device, predict_noise=config.predict_noise, noise_schedule="linear")

name = 'GCDiffuser'
agent.load(glob.glob(dir + '/' + name + '*' + env + '*' + 'latest.pt')[0])

# Environment info 
print('==== Environment Info ====')
print('Name:',config.env)
print('Obs Space:',obs_dim)
print('Act Space:',act_dim)
# main evaluation loop
print('==== Running Eval ====')
for task_id in [1,2,3,4,5]:
    print('== Task',task_id,' ==')
    envs.reset(options = {'task_id':task_id})
    avg_completion = dd_eval(agent,invdyn,envs,config)
    print('Avg. Completion:',avg_completion)
    